In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [4]:
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

LANGCHAIN_TRACING=os.getenv("LANGCHAIN_TRACING")
os.environ["LANGCHAIN_TRACING"] = LANGCHAIN_TRACING

LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT

LANGSMITH_PROJECT = "langchain_essentials"

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It")

In [6]:
model.invoke("Hello, how are you?")

AIMessage(content="As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you!\n\nHow can I help you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 15, 'total_tokens': 54, 'completion_time': 0.070909091, 'prompt_time': 0.00124623, 'queue_time': 0.020609877, 'total_time': 0.072155321}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--be9b70cb-0c75-48d7-825a-b35aad3330b1-0', usage_metadata={'input_tokens': 15, 'output_tokens': 39, 'total_tokens': 54})

In [7]:
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory


In [8]:
memory = ConversationSummaryMemory(llm=model,return_messages=True)

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_43654/25687835.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=model,return_messages=True)


In [9]:
memory.save_context(
    {"Human": "John and Jack are working on a new project on ML"},
    {"AI": "Thats interesting! What is their project about?"}
)

In [10]:
memory.load_memory_variables({})

{'history': [SystemMessage(content="Human: John and Jack are working on a new project on ML\nAI: That's interesting! What is their project about? \n\nNew summary: The human mentions that John and Jack are working on a new project related to machine learning. The AI expresses interest and asks for more details about the project. \n", additional_kwargs={}, response_metadata={})]}

In [11]:
summary = memory.load_memory_variables({})

In [12]:
print(summary["history"][0].content)

Human: John and Jack are working on a new project on ML
AI: That's interesting! What is their project about? 

New summary: The human mentions that John and Jack are working on a new project related to machine learning. The AI expresses interest and asks for more details about the project. 



In [13]:
memory.save_context(
    {"Human": "The project is about building a healthcare application."},
    {"AI": "Thats great! Is it for hospitals or patients?"}
)

memory.save_context(
    {"Human": "It is for the patients."},
    {"AI": "Cool, wishing Jack and John all the best"}
)

In [15]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human mentions that John and Jack are working on a new project related to machine learning. The AI expresses interest and asks for more details about the project. The human reveals that the project is focused on building a healthcare application for patients. The AI responds positively and wishes John and Jack all the best.  \n', additional_kwargs={}, response_metadata={})]}

In [16]:
memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='John and Jack are working on a new project on ML', additional_kwargs={}, response_metadata={}), AIMessage(content='Thats interesting! What is their project about?', additional_kwargs={}, response_metadata={}), HumanMessage(content='The project is about building a healthcare application.', additional_kwargs={}, response_metadata={}), AIMessage(content='Thats great! Is it for hospitals or patients?', additional_kwargs={}, response_metadata={}), HumanMessage(content='It is for the patients.', additional_kwargs={}, response_metadata={}), AIMessage(content='Cool, wishing Jack and John all the best', additional_kwargs={}, response_metadata={})])

In [17]:
memory.chat_memory.messages

[HumanMessage(content='John and Jack are working on a new project on ML', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Thats interesting! What is their project about?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='The project is about building a healthcare application.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Thats great! Is it for hospitals or patients?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='It is for the patients.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Cool, wishing Jack and John all the best', additional_kwargs={}, response_metadata={})]

In [18]:
messages = memory.chat_memory.messages

In [19]:
history = ChatMessageHistory()

In [20]:
history.add_user_message("Hello")

In [21]:
history.add_ai_message("Hi! How can I help you?")

In [22]:
new_memory = ConversationSummaryMemory.from_messages(
    llm=model,
    chat_memory=history,
    memory_key="summary",
    human_prefix="User",
    ai_prefix="Assistant"
)

In [23]:
new_memory.buffer

'Current summary: A user greets the assistant. \n\nNew lines of conversation:\nUser: Hello\nAssistant: Hi! How can I help you? \n\nNew summary: A user greets the assistant, and the assistant responds with a greeting and an offer to help. \n'

#### Chaining

In [24]:
from langchain.chains import ConversationChain